In [1]:
import pymysql
import math
import scipy.stats as stat
from scipy.stats import norm
import pandas as pd
import numpy as np
import urllib.request, json

In [2]:
#grab the entire table from sql
#contains recent traded option log and updated gex/vex/ddoi values
#make this take an argument in case we have more tables in the future
def get_sql_df():
    Host = 'deribit-db-instance.csduabs5w5go.us-east-1.rds.amazonaws.com'
    User = 'admin'
    Password = 'Deribit123'
    Database = 'deribit'
    con = pymysql.connect(user=User, password=Password,host=Host, database=Database, port=3306, ssl_key='../../Downloads/deribit-key-pair.pem')
    try:
        with con.cursor() as cur:
            query = 'SELECT * FROM deribit.option_table'
            cur.execute(query)
            df = pd.read_sql(query, con)
            return df
    finally:
        con.close()

In [3]:
df = get_sql_df()
df

,MyUnknownColumn,trade_seq,trade_id,timestamp,tick_direction,price,mark_price,iv,instrument_name,index_price,direction,amount,gex,vex,ddoi
0,0,3,64622797,1613793025351,0,0.1625,0.245046,107.69,BTC-26MAR21-54000-C,55753.42,buy,10,10,10,0
1,0,2,64649826,1613851501155,0,0.1340,0.073299,165.31,BTC-5MAR21-57000-P,56738.42,buy,3,20,15,0
2,0,50,64649954,1613851546122,0,0.1370,0.063862,198.70,BTC-5MAR21-57000-C,56731.02,buy,3,30,16,0
3,0,57,64650074,1613851603336,0,0.0695,0.061493,134.11,BTC-5MAR21-60000-C,56796.98,buy,3,40,20,0
4,0,336,64657047,1613855054463,2,0.0560,0.151207,39.36,BTC-26FEB21-54000-C,56715.78,sell,2,30,10,0
5,0,2,64657489,1613857287961,2,0.0100,0.003001,114.51,BTC-21FEB21-56000-P,56704.36,buy,1,10,20,0


In [4]:
df.shape

(6, 15)

In [5]:
new_trade = df.iloc[-1]
new_trade

MyUnknownColumn                      0
trade_seq                            2
trade_id                      64657489
timestamp                1613857287961
tick_direction                       2
price                             0.01
mark_price                  0.00300078
iv                              114.51
instrument_name    BTC-21FEB21-56000-P
index_price                    56704.4
direction                          buy
amount                               1
gex                                 10
vex                                 20
ddoi                                 0
Name: 5, dtype: object

In [6]:
testingdf = df.copy()
testingdf

,MyUnknownColumn,trade_seq,trade_id,timestamp,tick_direction,price,mark_price,iv,instrument_name,index_price,direction,amount,gex,vex,ddoi
0,0,3,64622797,1613793025351,0,0.1625,0.245046,107.69,BTC-26MAR21-54000-C,55753.42,buy,10,10,10,0
1,0,2,64649826,1613851501155,0,0.1340,0.073299,165.31,BTC-5MAR21-57000-P,56738.42,buy,3,20,15,0
2,0,50,64649954,1613851546122,0,0.1370,0.063862,198.70,BTC-5MAR21-57000-C,56731.02,buy,3,30,16,0
3,0,57,64650074,1613851603336,0,0.0695,0.061493,134.11,BTC-5MAR21-60000-C,56796.98,buy,3,40,20,0
4,0,336,64657047,1613855054463,2,0.0560,0.151207,39.36,BTC-26FEB21-54000-C,56715.78,sell,2,30,10,0
5,0,2,64657489,1613857287961,2,0.0100,0.003001,114.51,BTC-21FEB21-56000-P,56704.36,buy,1,10,20,0


In [7]:
#working function here
def get_instrument(all_trades):
    #drop inactive options
    for x in all_trades['instrument_name']:
        url = 'https://test.deribit.com/api/v2/public/get_instrument?instrument_name='
        url += x
        with urllib.request.urlopen(url) as url:
            data = json.loads(url.read().decode())
        result = data['result']
        if(result['is_active']):
            pass
        else:
            iname = result['instrument_name']
            print("iname: " , iname)
            inst_names = all_trades[all_trades['instrument_name'] == iname].index
            all_trades.drop(inst_names, inplace=True)
    #make buys positive 1 and sells -1 since we want to
    #apply appropriate weights on values when calculating GEX/VEX
    all_trades['direction'] = all_trades['direction'].replace('buy', 1)
    all_trades['direction'] = all_trades['direction'].replace('sell', -1)
    active_options = all_trades
    return active_options

In [8]:
no_stale = get_instrument(testingdf)

iname:  BTC-5MAR21-57000-P
iname:  BTC-5MAR21-57000-C
iname:  BTC-5MAR21-60000-C
iname:  BTC-26FEB21-54000-C
iname:  BTC-21FEB21-56000-P


In [9]:
no_stale

,MyUnknownColumn,trade_seq,trade_id,timestamp,tick_direction,price,mark_price,iv,instrument_name,index_price,direction,amount,gex,vex,ddoi
0,0,3,64622797,1613793025351,0,0.1625,0.245046,107.69,BTC-26MAR21-54000-C,55753.42,1,10,10,10,0


In [10]:
get_orderbook_url = "https://test.deribit.com/api/v2/public/get_order_book?depth=5&instrument_name=BTC-5MAR21-57000-C"
with urllib.request.urlopen(get_orderbook_url) as get_orderbook_url:
    ob_data = json.loads(get_orderbook_url.read().decode())
ob_data = pd.DataFrame.from_dict(ob_data['result'], orient='index')
ob_data = ob_data.transpose()
ob_data

,underlying_price,underlying_index,timestamp,stats,state,open_interest,min_price,max_price,mark_price,mark_iv,...,delivery_price,change_id,bids,bid_iv,best_bid_price,best_bid_amount,best_ask_price,best_ask_amount,asks,ask_iv
0,47155.7,SYN.BTC-5MAR21,1614931200012,"{'volume': 7.1, 'price_change': -50.0, 'low': ...",closed,0,0.0001,0.2,0,250,...,47191.3,5419243086,[],0,0,0,0,0,[],0


In [11]:
def get_orderbook_data(df_to_append_to, instrument_name):
    #use this to get relevant information from deribit: i.e greeks, open interest,& interest rate
    get_orderbook_url = "https://test.deribit.com/api/v2/public/get_order_book?depth=5&instrument_name="
    get_orderbook_url += instrument_name
    with urllib.request.urlopen(get_orderbook_url) as get_orderbook_url:
        orderbook_data = json.loads(get_orderbook_url.read().decode())
    orderbook_data = pd.DataFrame.from_dict(orderbook_data['result'], orient='index')
    orderbook_data = orderbook_data.transpose()
    combined_df = df_to_append_to.append(orderbook_data, ignore_index=True)
    return combined_df

In [13]:
def get_greeks(active_trades):
    greeks = pd.DataFrame()
    for i in active_trades['instrument_name']:
        greeks = get_orderbook_data(greeks, i)
    columns = greeks.loc[:, ['underlying_price','underlying_index', 'open_interest','interest_rate', 'mark_iv', 'greeks']]
    greeks = pd.DataFrame(columns['greeks'].values.tolist())
    columns = columns.drop(['greeks'], axis = 1)
    greeks = greeks.join(columns)
    greeks['vanna'] = np.nan
    return greeks

In [17]:
greeks = get_greeks(no_stale)

In [18]:
def get_gex(active_trades, active_trades_greeks):
    total_gamma = active_trades['amount'] * active_trades_greeks['gamma'] * active_trades['direction']
    return sum(total_gamma)

In [19]:
get_gex(no_stale, greeks)

0.0001

In [20]:
def get_vanna(active_trades, greeks):
    #get days till expiration from active trades and add as a series
    dte = pd.to_datetime(active_trades['instrument_name'].apply(lambda x: x.split('-')[1]))-pd.Timestamp.today()
    strike = active_trades['instrument_name'].apply(lambda x: x.split('-')[2]).astype(int)
    active_trades['strike'] = strike
    #active_trades.astype({'strike': 'int32'}).dtypes
    active_trades['dte'] = dte.dt.days
    sigmaT = greeks['mark_iv'] * active_trades['dte'] ** 0.5
    d1 = (math.log(greeks['underlying_price'] / active_trades['strike']) + \
                   (greeks['interest_rate'] * 0.5 * (greeks['mark_iv'] ** 2)) \
                   * active_trades['dte']) / sigmaT
    d2 = d1 - sigmaT
    vannas = []
    #we'd like to vectorize this but d1 norm.pdf(d1) is not trivial due to some stupid type error
    #I can't be bothered to figure out right now
    for i in range(active_trades.shape[0]):
        vannas.append(0.01 * -math.e ** active_trades['dte'][i] * d2[i] / greeks['mark_iv'][i] * norm.pdf(d1[i]))
    return vannas
    #[0.01 * -e ** (-self.q * self.T) * self.d2 / self.sigma * norm.pdf(self.d1)]

In [21]:
vanna = get_vanna(no_stale, greeks)

In [22]:
no_stale

,MyUnknownColumn,trade_seq,trade_id,timestamp,tick_direction,price,mark_price,iv,instrument_name,index_price,direction,amount,gex,vex,ddoi,strike,dte
0,0,3,64622797,1613793025351,0,0.1625,0.245046,107.69,BTC-26MAR21-54000-C,55753.42,1,10,10,10,0,54000,16


In [23]:
def get_vex(active_trades, greeks):
    vanna = get_vanna(active_trades, greeks)
    return vanna * no_stale['amount'] * no_stale['direction']

In [24]:
vex = get_vex(no_stale, greeks)

In [25]:
vex

0    1.418018e+06
dtype: float64